In [4]:
from openai import OpenAI

In [5]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [6]:
# Geopolitics Game

class GeopoliticsGame:
    """
    A game where 5 LLM players (representing countries) compete for territories.
    Each player starts with 10 territories. On their turn, a player proposes to take
    1 territory from another player. All players vote Yes or No. If more Yes votes,
    the territory transfer succeeds. Goal: have the most territories after X turns.
    """
    
    def __init__(self, player_models, country_names, ollama_client, starting_territories=10):
        """
        Initialize the game with 5 players.
        
        Args:
            player_models: List of 5 model names (e.g., ["phi3:medium", "llama2", ...])
            country_names: List of 5 country names (e.g., ["Avalon", "Borealis", ...])
            ollama_client: The Ollama OpenAI client instance
            starting_territories: Starting territories per player (default: 10)
        """
        if len(player_models) != 5:
            raise ValueError("Must have exactly 5 players")
        if len(country_names) != 5:
            raise ValueError("Must have exactly 5 country names")
        
        self.player_models = player_models
        self.country_names = country_names
        self.ollama = ollama_client
        self.starting_territories = starting_territories
        
        # Game state
        self.territories = [starting_territories] * 5  # Each player starts with 10
        self.history = []  # Full history of all turns
        self.total_tokens = 0
        
        # Create player info dictionary
        self.players = [
            {"id": i, "model": model, "country": country}
            for i, (model, country) in enumerate(zip(player_models, country_names))
        ]
    
    def build_game_state_context(self, player_id):
        """
        Build a context string showing current game state from a player's perspective.
        
        Args:
            player_id: The ID of the player (0-4)
        """
        context = f"You are {self.country_names[player_id]}, representing Player {player_id + 1}.\n\n"
        context += "Current territory counts:\n"
        for i, (country, territories) in enumerate(zip(self.country_names, self.territories)):
            if i == player_id:
                context += f"  {country} (YOU): {territories} territories\n"
            else:
                context += f"  {country}: {territories} territories\n"
        
        return context
    
    def build_history_context(self):
        """
        Build a context string from all previous turns.
        """
        if not self.history:
            return ""
        
        context = "\n\nFull game history:\n"
        for turn_num, turn_data in enumerate(self.history, 1):
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            votes = turn_data['votes']
            passed = turn_data['passed']
            
            context += f"\nTurn {turn_num}: {proposer} proposed to take 1 territory from {target}.\n"
            context += "Votes: "
            vote_details = []
            for i, (country, vote) in enumerate(zip(self.country_names, votes)):
                vote_details.append(f"{country}: {vote}")
            context += ", ".join(vote_details) + "\n"
            
            if passed:
                context += f"Result: PASSED. {proposer} took 1 territory from {target}.\n"
            else:
                context += f"Result: FAILED. No territory transfer.\n"
        
        return context
    
    def get_proposal(self, player_id):
        """
        Query a player to propose which country to take a territory from.
        
        Args:
            player_id: The ID of the proposing player (0-4)
            
        Returns:
            target_id: The ID of the target country (0-4, not equal to player_id)
        """
        player = self.players[player_id]
        game_state = self.build_game_state_context(player_id)
        history = self.build_history_context()
        
        prompt = f"""{game_state}{history}

It is your turn. You must propose to take 1 territory from one of the other countries.
Which country do you want to take a territory from?

Respond with only the country name (e.g., "Avalon" or "Borealis").
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            choice = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Find matching country name
            choice_lower = choice.lower()
            for i, country in enumerate(self.country_names):
                if i != player_id and (country.lower() in choice_lower or choice_lower in country.lower()):
                    return i
            
            # If no match found, default to first available country
            print(f"Warning: Could not parse '{choice}' from {player['country']}, defaulting to first available")
            for i in range(5):
                if i != player_id:
                    return i
            
            return (player_id + 1) % 5  # Fallback
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            # Default to first available country
            for i in range(5):
                if i != player_id:
                    return i
            return (player_id + 1) % 5
    
    def get_vote(self, player_id, proposer_id, target_id):
        """
        Query a player to vote Yes or No on a proposal.
        
        Args:
            player_id: The ID of the voting player (0-4)
            proposer_id: The ID of the player making the proposal (0-4)
            target_id: The ID of the target country (0-4)
            
        Returns:
            "Yes" or "No" (normalized)
        """
        player = self.players[player_id]
        proposer = self.country_names[proposer_id]
        target = self.country_names[target_id]
        game_state = self.build_game_state_context(player_id)
        history = self.build_history_context()
        
        prompt = f"""{game_state}{history}

{proposer} has proposed to take 1 territory from {target}.
Do you vote Yes or No on this proposal?

Respond with only "Yes" or "No".
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            choice = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Normalize the response
            choice_lower = choice.lower()
            if "yes" in choice_lower or "vote yes" in choice_lower:
                return "Yes"
            elif "no" in choice_lower or "vote no" in choice_lower:
                return "No"
            else:
                # Default to No if unclear
                print(f"Warning: Unclear response '{choice}' from {player['country']}, defaulting to No")
                return "No"
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            return "No"  # Default to No on error
    
    def play_turn(self, turn_num):
        """
        Play a single turn of the game.
        
        Args:
            turn_num: The turn number
        """
        # Determine whose turn it is (round-robin: turn 1 = player 0, turn 2 = player 1, etc.)
        proposer_id = (turn_num - 1) % 5
        proposer = self.players[proposer_id]
        
        print(f"\n{'='*60}")
        print(f"Turn {turn_num}: {proposer['country']}'s turn")
        print(f"{'='*60}")
        
        # Get proposal
        print(f"Querying {proposer['country']} ({proposer['model']}) for proposal...")
        target_id = self.get_proposal(proposer_id)
        target = self.country_names[target_id]
        print(f"{proposer['country']} proposes to take 1 territory from {target}")
        
        # Get votes from all players
        print(f"\nCollecting votes...")
        votes = []
        for i in range(5):
            vote = self.get_vote(i, proposer_id, target_id)
            votes.append(vote)
            print(f"  {self.country_names[i]}: {vote}")
        
        # Count votes
        yes_count = sum(1 for v in votes if v == "Yes")
        no_count = sum(1 for v in votes if v == "No")
        passed = yes_count > no_count
        
        # Execute territory transfer if passed
        if passed:
            if self.territories[target_id] > 0:
                self.territories[proposer_id] += 1
                self.territories[target_id] -= 1
                print(f"\n✓ Proposal PASSED ({yes_count} Yes, {no_count} No)")
                print(f"  {proposer['country']} now has {self.territories[proposer_id]} territories")
                print(f"  {target} now has {self.territories[target_id]} territories")
            else:
                print(f"\n✗ Proposal PASSED but {target} has no territories to lose!")
        else:
            print(f"\n✗ Proposal FAILED ({yes_count} Yes, {no_count} No)")
            print(f"  No territory transfer")
        
        # Store turn data
        turn_data = {
            'turn': turn_num,
            'proposer_id': proposer_id,
            'target_id': target_id,
            'votes': votes,
            'yes_count': yes_count,
            'no_count': no_count,
            'passed': passed,
            'territories_after': self.territories.copy()
        }
        self.history.append(turn_data)
        
        # Display current standings
        print(f"\nCurrent standings:")
        for i, (country, territories) in enumerate(zip(self.country_names, self.territories)):
            print(f"  {country}: {territories} territories")
    
    def play_game(self, num_turns):
        """
        Play a full game with the specified number of turns.
        
        Args:
            num_turns: Number of turns to play
        """
        print(f"\n{'='*60}")
        print(f"Starting Geopolitics Game")
        print(f"{'='*60}")
        print(f"\nPlayers:")
        for i, player in enumerate(self.players):
            print(f"  Player {i+1}: {player['country']} ({player['model']})")
        print(f"\nStarting territories per player: {self.starting_territories}")
        print(f"Total turns: {num_turns}")
        print(f"{'='*60}")
        
        # Reset game state
        self.territories = [self.starting_territories] * 5
        self.history = []
        self.total_tokens = 0
        
        # Play turns
        for turn_num in range(1, num_turns + 1):
            self.play_turn(turn_num)
        
        # Final summary
        self.print_summary()
    
    def print_summary(self):
        """Print a summary of the game."""
        print(f"\n{'='*60}")
        print("FINAL RESULTS")
        print(f"{'='*60}")
        
        # Sort players by territories
        standings = sorted(
            [(self.country_names[i], self.territories[i], i) for i in range(5)],
            key=lambda x: x[1],
            reverse=True
        )
        
        print(f"\nFinal standings:")
        for rank, (country, territories, player_id) in enumerate(standings, 1):
            print(f"  {rank}. {country}: {territories} territories")
        
        winner = standings[0]
        print(f"\n🏆 Winner: {winner[0]} with {winner[1]} territories!")
        
        print(f"\nTurn-by-turn summary:")
        for turn_data in self.history:
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            result = "PASSED" if turn_data['passed'] else "FAILED"
            print(f"  Turn {turn_data['turn']}: {proposer} → {target} ({result}, {turn_data['yes_count']}-{turn_data['no_count']})")
        
        print(f"\nTotal tokens used: {self.total_tokens}")

In [ ]:
# Create and play the Geopolitics game
# You need 5 models and 5 country names
# Each player starts with 10 territories (default)
# Players take turns proposing to take territories from others
# All players vote Yes/No, and if more Yes votes, the transfer succeeds

game = GeopoliticsGame(
    player_models=[
        "phi3:medium",    # Change to your preferred models
        "phi3:medium",
        "phi3:medium",
        "phi3:medium",
        "phi3:medium"
    ],
    country_names=["Avalon", "Borealis", "Crestfall", "Dawnhold", "Elysium"],  # Change country names if you want
    ollama_client=ollama,
    starting_territories=10  # Each player starts with 10 territories
)

# Play 5 rounds (you can change this number)
game.play_game(num_turns=10)


Starting Geopolitics Game

Players:
  Player 1: Avalon (phi3:medium)
  Player 2: Borealis (phi3:medium)
  Player 3: Crestfall (phi3:medium)
  Player 4: Dawnhold (phi3:medium)
  Player 5: Elysium (phi3:medium)

Starting territories per player: 10
Total turns: 50

Turn 1: Avalon's turn
Querying Avalon (phi3:medium) for proposal...
Avalon proposes to take 1 territory from Borealis

  Avalon: No
  Borealis: No
  Crestfall: No
  Dawnhold: Yes
  Elysium: No

✗ Proposal FAILED (1 Yes, 4 No)
  No territory transfer

Current standings:
  Avalon: 10 territories
  Borealis: 10 territories
  Crestfall: 10 territories
  Dawnhold: 10 territories
  Elysium: 10 territories

Turn 2: Borealis's turn
Querying Borealis (phi3:medium) for proposal...
Borealis proposes to take 1 territory from Elysium

  Avalon: No
  Borealis: Yes
  Crestfall: No
  Dawnhold: No
  Elysium: No

✗ Proposal FAILED (1 Yes, 4 No)
  No territory transfer

Current standings:
  Avalon: 10 territories
  Borealis: 10 territories
  Cre

In [ ]:
Final standings:
  1. Crestfall: 11 territories
  2. Dawnhold: 11 territories
  3. Elysium: 11 territories
  4. Borealis: 10 territories
  5. Avalon: 7 territories

🏆 Winner: Crestfall with 11 territories!

Turn-by-turn summary:
  Turn 1: Avalon → Crestfall (FAILED, 1-4)
  Turn 2: Borealis → Avalon (FAILED, 2-3)
  Turn 3: Crestfall → Elysium (FAILED, 1-4)
  Turn 4: Dawnhold → Avalon (FAILED, 2-3)
  Turn 5: Elysium → Dawnhold (FAILED, 2-3)
  Turn 6: Avalon → Borealis (PASSED, 3-2)
  Turn 7: Borealis → Avalon (PASSED, 3-2)
  Turn 8: Crestfall → Avalon (PASSED, 5-0)
  Turn 9: Dawnhold → Avalon (PASSED, 4-1)
  Turn 10: Elysium → Avalon (PASSED, 4-1)